# **Context Managers in Python**

#### 💡Context managers are a feature of Python that allows for the setup and teardown of resources automatically. 

They are often used to manage resources such as 
- file streams
- locks
- database connections 

in a safe, clean, and concise manner.

## What is a Context Manager?

#### 💡 A context manager in Python is an object designed to be used in a **`with`** statement, which guarantees that resources are properly acquired and released, even if an error occurs during their use.

## The **`with`** Statement

The **`with`** statement simplifies exception handling by encapsulating common preparation and cleanup tasks in so-called **context managers**.

It ensures that the necessary "setup" and "cleanup" actions are taken, regardless of whether the block of code executed successfully or raised an exception.


#### 💡 The **`with`** statement is used to wrap the execution of a block of code. It ensures that predefined cleanup actions are performed after the block is executed, which is useful for resource management tasks like opening and closing files.

### Working with a Context Manager

- **`contextlib.closing`**: This is a helper function that returns a context manager for objects that do not support the context management protocol but have a **`close()`** method.

In [ ]:
from contextlib import closing
from urllib.request import urlopen

url = "http://www.python.org"
with closing(urlopen(url)) as page:
    for line in page:
        print(line)

- **`__enter__` and `__exit__` Methods**: Custom context managers implement these two methods. The **`__enter__`** method is run at the beginning of the **`with`** block, and **`__exit__`** is run at the end, handling resource cleanup

In [ ]:
class ManagedFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

with ManagedFile('hello.txt') as f:
    f.write('Hello, world!')

### Utilizing **`contextlib.contextmanager`**

This is a decorator that allows a simple function with a **`yield`** statement to be used as a context manager, without the need to define a class that implements the **`__enter__`** and **`__exit__`** methods.

In [ ]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

with managed_file('hello.txt') as f:
    f.write('Hello, world!')

### Custom Context Managers: Classes and Generators

- **Classes**: By defining **`__enter__`** and **`__exit__`** methods in a class, you can create a context manager that can execute setup code before the **`with`** block and cleanup code afterward.
- **Generators**: Using **`contextlib.contextmanager`**, you can write a generator function that yields control back to the **`with`** block and ensures that cleanup code runs after the block exits.
    
    ```python

    ```
    

    


   

In [ ]:
import os
from contextlib import contextmanager

@contextmanager
def change_dir(destination):
    try:
        cwd = os.getcwd()
        os.chdir(destination)
        yield
    finally:
        os.chdir(cwd)

# Use this context manager to temporarily change the working directory.

### **More Examples**

In [ ]:
class MockDatabase:
    def __enter__(self):
        print("Connecting to the database...")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Disconnecting from the database...")

# Simulate opening and closing a database connection.

In [ ]:
@contextmanager
def logging_context():
    print("Entering context")
    yield
    print("Exiting context")

# Demonstrate entering and exiting a block with logging.

## Context Manager Exercises

### Exercise 1
### Context Manager for Timing a List Comprehension

Problem: Use a Context Manager from `contextlib` in order to time the runtime of a list comprehension command.

In [ ]:
from contextlib import contextmanager
import time

@contextmanager
def timing():
    start = time.time()
    yield
    end = time.time()
    print(f"Time taken: {end - start}s")

with timing():
    [x**2 for x in range(1000000)]

### Exercise 2
### Context Manager for Opening Multiple Files 
Build a context manager to open multiple files

In [ ]:
from contextlib import contextmanager

@contextmanager
def open_multiple_files(*files, mode='r'):
    opened_files = [open(file, mode) for file in files]
    try:
        yield opened_files
    finally:
        for f in opened_files:
            f.close()

with open_multiple_files('file1.txt', 'file2.txt', mode='w') as files:
    files[0].write('Hello')
    files[1].write('World')


### Exercise 3: Context Manager for a Temporary Change in Working Directory

In [ ]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(destination):
    try:
        cwd = os.getcwd()
        os.chdir(destination)
        yield
    finally:
        os.chdir(cwd)

with change_directory("/tmp"):
    print("Current Working Directory:", os.getcwd())


### Exercise 4: Suppressing Specific Exceptions
Make a Context Manager to suppress `FileNotFoundError` allowing the code to continue if the file doesn't exist.

Import `suppress` from `contextlib` library

In [ ]:
from contextlib import suppress

# Suppresses FileNotFoundError, allowing the code to continue if the file doesn't exist.
with suppress(FileNotFoundError):
    os.remove('non_existent_file.txt')
print("Continuing execution.")


## Advanced Exercises

### Exercise 5: Resource Pool
Implement a context manager for managing resources from a pool.

In [ ]:
# Implement a context manager for managing resources from a pool.

from contextlib import contextmanager
import queue

class Pool:
    def __init__(self, resources):
        self.available_resources = queue.Queue()
        for resource in resources:
            self.available_resources.put(resource)

    @contextmanager
    def get_resource(self):
        resource = self.available_resources.get()
        try:
            yield resource
        finally:
            self.available_resources.put(resource)

# Example usage
pool = Pool(["resource1", "resource2", "resource3"])
with pool.get_resource() as res:
    print(f"Using {res}")


### Exercise 6: Timer Context Manager
Create a context manager that measures the execution time of a code block.



In [ ]:
# Create a context manager that measures the execution time of a code block.

from contextlib import contextmanager
import time

@contextmanager
def timer():
    start = time.time()
    yield
    end = time.time()
    print(f"Elapsed time: {end - start} seconds")

# Example usage
with timer():
    sum(range(1000000))


### Exercise 7: Context Manager for Locking
Develop a context manager that acquires and releases a lock.


In [ ]:
# Develop a context manager that acquires and releases a lock.

from threading import Lock
from contextlib import contextmanager

lock = Lock()

@contextmanager
def acquire_lock(l):
    l.acquire()
    try:
        yield
    finally:
        l.release()

# Example usage
with acquire_lock(lock):
    # perform thread-safe operations
    print("Lock acquired")


### Exercise 8 Redirecting Standard Output
Write a context manager that temporarily redirects `sys.stdout`.


In [ ]:
# Write a context manager that temporarily redirects `sys.stdout`.

from contextlib import contextmanager
import sys

@contextmanager
def redirect_stdout(new_target):
    old_target = sys.stdout
    sys.stdout = new_target
    try:
        yield new_target
    finally:
        sys.stdout = old_target

# Example usage
with open('log.txt', 'w') as f, redirect_stdout(f):
    print("This will go to 'log.txt'")


### Exercise 9: Atomic File Write
Design a context manager that ensures atomic write operations to a file.


In [ ]:
# Design a context manager that ensures atomic write operations to a file.
from contextlib import contextmanager
import os

@contextmanager
def atomic_write(file_name, mode='w'):
    temp_file = file_name + '.tmp'
    try:
        with open(temp_file, mode) as f:
            yield f
        os.replace(temp_file, file_name)
    except:
        os.remove(temp_file)
        raise

# Example usage
with atomic_write('data.txt') as f:
    f.write("Safe writing")


### Exercise 10: Context Manager for Database Transaction
Write a class `Database` that contains `connect()`, `disconnect()`, `start_transaction`, `end_transaction` methods 

Use `contextlib` from `contextmanager` to manage database transaction


In [1]:
class Database:
    def __init__(self):
        self.connected = False
        self.transaction_started = False

    def connect(self):
        self.connected = True

    def disconnect(self):
        self.connected = False

    def start_transaction(self):
        if self.connected:
            self.transaction_started = True
            print("Transaction started")

    def end_transaction(self):
        if self.transaction_started:
            print("Transaction ended")
            self.transaction_started = False

from contextlib import contextmanager

@contextmanager
def database_transaction(db):
    db.connect()
    db.start_transaction()
    try:
        yield
    finally:
        db.end_transaction()
        db.disconnect()

db = Database()
with database_transaction(db):
    print("Performing transaction...")


Transaction started
Performing transaction...
Transaction ended
